In [1]:
import os
import glob
import codecs
import spacy
import json
import pandas as pd
import numpy as np
import networkx as nx

import nltk
from nltk.tokenize import sent_tokenize
import spacy
from spacy import displacy
from collections import Counter

from pathlib import Path
from pyspark import SparkContext
from afinn import Afinn
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
import en_core_web_sm

nlp = en_core_web_sm.load()

In [3]:
import re

In [4]:
def flatten(input_list):
    '''
    A function to flatten complex list.
    :param input_list: The list to be flatten
    :return: the flattened list.
    '''

    flat_list = []
    for i in input_list:
        if type(i) == list:
            flat_list += flatten(i)
        else:
            flat_list += [i]

    return flat_list


def name_entity_recognition(sentence,names_list):
    '''
    A function to retrieve name entities in a sentence.
    :param sentence: the sentence to retrieve names from.
    :return: a name entity list of the sentence.
    '''

    doc = nlp(sentence)
    # retrieve person and organization's name from the sentence
    name_entity = [x for x in doc.ents if x.label_ in ['PERSON', 'ORG']]
    # convert all names to lowercase and remove 's in names
    name_entity = [str(x).lower().replace("'s","") for x in name_entity]
    # split names into single words ('Harry Potter' -> ['Harry', 'Potter'])
    name_entity = [x.split(' ') for x in name_entity]
    # flatten the name list
    name_entity = flatten(name_entity)
    # remove name words that are less than 3 letters to raise recognition accuracy
    name_entity = [x for x in name_entity if len(x) >= 3]
    # remove name words that are in the set of 4000 common words
    name_entity = [x for x in name_entity if x not in common_words]
    # remove name words that are in the set of races
    name_entity = [x for x in name_entity if x not in races]
    #print(name_entity)
    for name in name_entity:
        names_list.append(name)
    return name_entity

In [23]:
sent_file = open(r'C:\Users\NYashch\nyashch-projects\all_sentences.txt','r',encoding = 'utf-8')
all_sentences = [line.strip() for line in sent_file]
sent_file.close()
tolkien_text = ''.join(all_sentences)

In [26]:
print(all_sentences[5500:5550])

['', '', '', '', '', '', '', '', 'OF THE RUIN OF BELERIAND 185', "before the face of his captains; for the rocks rang with the shrill music of Fingolfin's horn, and his voice came keen and clear down into the depths of Angband; and Fingolfin named Morgoth craven, and lord of slaves.", 'Therefore Morgoth came, climbing slowly from his subterranean throne, and the rumour of his feet was like thunder underground.', 'And he issued forth clad in black armour; and he stood before the King like a tower, iron-crowned, and his vast shield, sable un-blazoned, cast a shadow over him like a stormcloud.', 'But Fingolfin gleamed beneath it as a star; for his mail was overlaid with silver, and his blue shield was set with crystals; and he drew his sword Ringil, that glittered like ice.', 'Then Morgoth hurled aloft Grond, the Hammer of the Underworld, and swung it down like a bolt of thunder.', 'But Fingolfin sprang aside, and Grond rent a mighty pit in the earth, whence smoke and fire darted.', 'Many

In [37]:
'''
tolkien_text_new = []
for sent in tolkien_text.split('.'):
    list_sent = sent.split()
    if 'Mithrandir' in list_sent:
        new_sent = sent.replace('Mithrandir', 'gendalf')
        tolkien_text_new.append(new_sent)
    elif 'Strider' in list_sent:
        new_sent = sent.replace('Strider', 'aragorn')
        tolkien_text_new.append(new_sent)
    elif 'Gamgee' in list_sent:
        new_sent = sent.replace('Gamgee', 'sam')
        tolkien_text_new.append(new_sent)
    elif 'Meriadoc' in list_sent:
        new_sent = sent.replace('Meriadoc', 'merry')
        tolkien_text_new.append(new_sent)
    elif 'Peregrin' in list_sent:
        new_sent = sent.replace('Peregrin', 'pippin')
        tolkien_text_new.append(new_sent)
    else:
        tolkien_text_new.append(new_sent)
'''

In [47]:
'''names = []
f = open(r'C:\Users\NYashch\nyashch-projects\my_names.txt','r',encoding = 'utf-8')
for elem in f.readlines():
    elem = elem.replace("\n","")
    names.append(elem)
'''

In [7]:
import re 
my_hobbits = open(r'C:\Users\NYashch\nyashch-projects\Diploma\my_hobbits.txt','r',encoding = 'utf-8')
hobbits = []
for elem in my_hobbits.readlines():
    if len(elem) > 4:
        elem = elem.replace("\n","")
        elem = elem.replace("\r","")
        hobbits.append(elem.lower().split()[0])

d_hobbits = dict.fromkeys(hobbits, 'hobbit')

my_dwarves = open(r'C:\Users\NYashch\nyashch-projects\Diploma\my_dwarves.txt','r',encoding = 'utf-8')
dwarves = []

for elem in my_dwarves.readlines():
    if len(elem) > 4:
        elem = elem.replace("\n","")
        elem = elem.replace("\r","")
        dwarves.append(elem.lower().split()[0])
d_dwarves = dict.fromkeys(dwarves, 'dwarf')
my_elves = open(r'C:\Users\NYashch\nyashch-projects\Diploma\my_elves.txt','r',encoding = 'utf-8')
elves = []
for elem in my_elves .readlines():
    if len(elem) > 4:
        elem = elem.replace("\n","")
        elem = elem.replace("\r","")
        elves.append(elem.lower().split()[0])
d_elves = dict.fromkeys(elves, 'elf')
my_maiar = open(r'C:\Users\NYashch\nyashch-projects\Diploma\my_maiar.txt','r',encoding = 'utf-8')
maiar = []
for elem in my_maiar.readlines():
    if len(elem) > 4:
        elem = elem.replace("\n","")
        elem = elem.replace("\r","")
        maiar.append(elem.lower().split()[0])
d_maiar = dict.fromkeys(maiar, 'maiar')
my_orcs = open(r'C:\Users\NYashch\nyashch-projects\Diploma\my_orcs.txt','r',encoding = 'utf-8')
orcs = []
for elem in my_orcs.readlines():
    if len(elem) > 4:
        elem = elem.replace("\n","")
        elem = elem.replace("\r","")
        orcs.append(elem.lower().split()[0])
d_orcs = dict.fromkeys(orcs, 'orc')
my_men = open(r'C:\Users\NYashch\nyashch-projects\Diploma\my_men.txt','r',encoding = 'utf-8')
men = []
for elem in my_men.readlines():
    if len(elem) > 4:
        elem = elem.replace("\n","")
        elem = elem.replace("\r","")
        men.append(elem.lower().split()[0])
d_men = dict.fromkeys(men, 'men')
my_valars = open(r'C:\Users\NYashch\nyashch-projects\Diploma\my_valars.txt','r',encoding = 'utf-8')
valars = []
for elem in my_valars.readlines():
    if len(elem) > 4:
        elem = elem.replace("\n","")
        elem = elem.replace("\r","")
        valars.append(elem.lower())
d_valars = dict.fromkeys(valars, 'valar')
ainur = ["ainur",'aluin','danuin','fanuin','morgoth','ranuin']
d_ainur = dict.fromkeys(ainur, 'ainur')
barlogs = ['gothmog','lungorthin','osombauko']
d_barlogs = dict.fromkeys(barlogs, 'barlog')
dragons = ['ancalagon','chrysophylax','glaurung','gostir','scatha','smaug','urulókë']
d_dragons = dict.fromkeys(dragons, 'dragon')
eagles = ['gwaihir','landroval','meneldor','thornhoth','thorondor','windlord']
d_eagles = dict.fromkeys(eagles, 'eagle')
ents = ['beechbone','entings','entmaidens','entmoot','entwives','fimbrethil','leaflock','onodrim','quickbeam','skinbark','treebeard']
d_ents = dict.fromkeys(ents, 'ent')
half_elven = ['ardamir eärendil','arwen','dior','eärendil','elladan','elrohir','elrond','elros','elurín','elwing','undómiel']
d_half_elven = dict.fromkeys(half_elven, 'half_elf')
werewolves = ['carcharoth','draugluin','thû']
d_werewolves = dict.fromkeys(werewolves, 'werewolf')
trolls = ['bert','olog-hai','tom','william']
d_trolls = dict.fromkeys(trolls, 'troll')


In [8]:
all_names = hobbits + dwarves +elves+maiar+orcs+men+valars+ainur+barlogs+dragons+eagles+ents+half_elven+werewolves+trolls
print('15 рас')

15 рас


In [9]:
names = list(set(all_names))

In [10]:
print(len(names))

517


In [17]:
print(all_sentences[140])

In [11]:

names.append('sam')

In [12]:
print(len(names))

518


In [27]:
color_map = {}
for name in names:
    if name in d_hobbits:
        color_map[name] ='#FFE4B5'
    elif name in d_elves:
        color_map[name] ='#00FF00'
    elif name in  d_dwarves:
        color_map[name] ='#808000'
    elif name in  d_trolls:
        color_map[name] ='#B8860B'
    elif name in  d_werewolves:
        color_map[name] ='#DCDCDC'
    elif name in  d_ents:
        color_map[name] ='#228B22'
    elif name in  d_half_elven:
        color_map[name] ='#98FB98'
    elif name in  d_eagles:
        color_map[name] ='#4B0082'
    elif name in  d_dragons:
        color_map[name] ='#800080'
    elif name in  d_barlogs:
        color_map[name] ='#800000'
    elif name in  d_ainur:
        color_map[name] ='#00FFFF'
    elif name in  d_valars:
        color_map[name] ='#FFFFF0'
    elif name in  d_orcs:
        color_map[name] ='#D2B48C'
    elif name in  d_men:
        color_map[name] ='#008080'
    elif name in  d_maiar:
        color_map[name] ='#000080'
    elif name == 'manwë':
        color_map[name] ='#FFFFF0'
    elif name == 'sam':
        color_map[name] ='#FFE4B5'
    elif name == 'melkor':
        color_map[name] ='#000080'
    elif name =='pippin':
        color_map[name] ='#FFE4B5'
    elif name == 'arda':
        color_map[name] ='#000080'

    else:
        names.remove(name)
 


In [28]:
cm_file = open(r'C:\Users\NYashch\nyashch-projects\Diploma\common_words.txt', 'r')
cm_lines = [line.strip() for line in cm_file]
cm_file.close()
common_words = []
for word in cm_lines:
    all_cm = word.split(',')
    for cm in all_cm:
           common_words.append(cm)
common_words.append('the')
races = ['trolls','troll', 'werewolves','werewolf', 'half_elven','elf', 'ents','ent', 'eagles','eagle','dragons','dragon', 'barlogs','barlog','ainur','hobbit','hobbits','dwarves','dwarf', 'elves', 'maiar', 'orcs', 'orc', 'valar','valars']

In [29]:
ner_sentences = []
for sent in all_sentences:
    ners = []
    for name in names:
        if name in sent.lower().split():
            ners.append(name)
    if len(ners)>1: # был 0
        ner_sentences.append(sent)

In [30]:
print(len(all_sentences))
print(len(ner_sentences))

156482
17286


In [31]:
'''color_map = []
for name in names:
    if name in d_hobbits:
        color_map.append('#FFE4B5')
    elif name in d_elves:
        color_map.append('#00FF00')
    elif name in  d_dwarves:
        color_map.append('#808000')
    elif name in  d_trolls:
        color_map.append('#B8860B')
    elif name in  d_werewolves:
        color_map.append('#DCDCDC')
    elif name in  d_ents:
        color_map.append('#228B22')
    elif name in  d_half_elven:
        color_map.append('#98FB98')
    elif name in  d_eagles:
        color_map.append('#4B0082')
    elif name in  d_dragons:
        color_map.append('#800080')
    elif name in  d_barlogs:
        color_map.append('#800000')
    elif name in  d_ainur:
        color_map.append('#00FFFF')
    elif name in  d_valars:
        color_map.append('#FFFFF0')
    elif name in  d_orcs:
        color_map.append('#D2B48C')
    elif name in  d_men:
        color_map.append('#008080')
    elif name in  d_maiar:
        color_map.append('#000080')
    elif name == 'manwë':
        color_map.append('#FFFFF0')
    elif name == 'sam':
        color_map.append('#FFE4B5')
    elif name == 'melkor':
        color_map.append('#000080')
    elif name =='pippin':
        color_map.append('#FFE4B5')
    else:
        color_map.append('#000000')
        names.remove(name)
'''

"color_map = []\nfor name in names:\n    if name in d_hobbits:\n        color_map.append('#FFE4B5')\n    elif name in d_elves:\n        color_map.append('#00FF00')\n    elif name in  d_dwarves:\n        color_map.append('#808000')\n    elif name in  d_trolls:\n        color_map.append('#B8860B')\n    elif name in  d_werewolves:\n        color_map.append('#DCDCDC')\n    elif name in  d_ents:\n        color_map.append('#228B22')\n    elif name in  d_half_elven:\n        color_map.append('#98FB98')\n    elif name in  d_eagles:\n        color_map.append('#4B0082')\n    elif name in  d_dragons:\n        color_map.append('#800080')\n    elif name in  d_barlogs:\n        color_map.append('#800000')\n    elif name in  d_ainur:\n        color_map.append('#00FFFF')\n    elif name in  d_valars:\n        color_map.append('#FFFFF0')\n    elif name in  d_orcs:\n        color_map.append('#D2B48C')\n    elif name in  d_men:\n        color_map.append('#008080')\n    elif name in  d_maiar:\n        colo

In [32]:
vect = CountVectorizer(vocabulary=names, stop_words='english')
name_frequency = vect.fit_transform([tolkien_text.lower()])
name_frequency = pd.DataFrame(name_frequency.toarray(), columns=vect.get_feature_names())
name_frequency = name_frequency.T
name_frequency = name_frequency.sort_values(by=0, ascending=False)
#name_frequency = name_frequency[0:40]
names = list(name_frequency.index)
name_frequency = list(name_frequency[0])


In [33]:
import pandas
df = pd.DataFrame()

In [51]:
'''name_vect = CountVectorizer(vocabulary=names, binary=True)
occurrence_each_sentence = name_vect.fit_transform(all_sentences).toarray()
cooccurrence_matrix = np.dot(occurrence_each_sentence.T, occurrence_each_sentence)
cooccurrence_matrix = np.tril(cooccurrence_matrix)
shape = cooccurrence_matrix.shape[0]
cooccurrence_matrix[[range(shape)], [range(shape)]] = 0
'''

In [34]:
from afinn import Afinn
afinn = Afinn()

In [35]:
d = {}

In [36]:
counter = 0
for sent in ner_sentences:
    counter += 1
    if counter % 1000 == 0:
        print(counter)
    sent_names = []
    for name in names:
        if name in sent.lower().split():
            sent_names.append(name)
    #name_entity_recognition(sent,sent_names)
    if len(sent_names) > 1:
        for name1 in sent_names:
            for name2 in sent_names:
                if name1 != name2:
                    key = ""
                    if name1 < name2:
                        key = name1 + "#" + name2
                    else:
                        key = name2 + "#" + name1
                    if d.get(key) == None:
                        d[key] =afinn.score(sent)
                    else:
                        d[key] = d[key]+afinn.score(sent)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000


In [37]:
items_list= []
for i in d.items():
    l = []
    line = list(i)[0].split('#')
    for elem in line:
        l.append(elem)
    l.append(list(i)[1])
    items_list.append(l)

In [38]:
sentiment_counter = -1

In [39]:
sentiment_edge_colors = []
for i in items_list:
    if i[2] < -5:
        sentiment_edge_colors.append('red')
    if i[2] > 2:
        sentiment_edge_colors.append('green')
    else:
        sentiment_edge_colors.append('blue')
names1 = []
names2 = []
edge_colors = []

for i in items_list:
    sentiment_counter += 1
    names1.append(i[0])
    names2.append(i[1])
    edge_colors.append(sentiment_edge_colors[sentiment_counter])

In [40]:
label_colors = []
for name in names1:
        label_colors.append(color_map[name])
    

In [214]:
'''just_colors = []
trird = []

for name in names:
    all_neigbours = []
    for pair in half_dat:
        if name in pair:
            if name == pair[0]:
                all_neigbours.append(name)
                all_neigbours.append(pair[1])
    just_colors.append(pair[2])
    trird.append(all_neigbours)
'''

'just_colors = []\ntrird = []\n\nfor name in names:\n    all_neigbours = []\n    for pair in half_dat:\n        if name in pair:\n            if name == pair[0]:\n                all_neigbours.append(name)\n                all_neigbours.append(pair[1])\n    just_colors.append(pair[2])\n    trird.append(all_neigbours)\n'

In [215]:
'''edges_list = []
for pair in half_dat:
    one_edge = []
    e0 = pair[0]
    e0 = '\"' + e0 + '\"'
    e1 = pair[1]
    e1 = '\"' + e1 + '\"'
    one_edge.append(e0)
    one_edge.append(e1)
    elem = ','.join(one_edge)
    edges_list.append(elem)
#print(edges_list)
'''

'edges_list = []\nfor pair in half_dat:\n    one_edge = []\n    e0 = pair[0]\n    e0 = \'"\' + e0 + \'"\'\n    e1 = pair[1]\n    e1 = \'"\' + e1 + \'"\'\n    one_edge.append(e0)\n    one_edge.append(e1)\n    elem = \',\'.join(one_edge)\n    edges_list.append(elem)\n#print(edges_list)\n'

In [216]:
'''itog = []
for i in names1:
    for i in names2:
        itog.append('\"'+i+'\"')
print(','.join(itog))
'''

'itog = []\nfor i in names1:\n    for i in names2:\n        itog.append(\'"\'+i+\'"\')\nprint(\',\'.join(itog))\n'

In [217]:
print(len(color_map))
print(len(names))
print(len(names1))
print(len(names2))
print(len(label_colors))

499
499
872
872
872


In [218]:
print(len(edge_colors))

872


In [41]:
data = {'name1': names1,'name2': names2,'label_color': label_colors,'edge_color': edge_colors}

data_df = pd.DataFrame.from_dict(data)

In [42]:
data_df

,name1,name2,label_color,edge_color
0,ainur,the,#00FFFF,green
1,great,the,#D2B48C,green
2,melkor,the,#FFFFF0,red
3,ainur,melkor,#00FFFF,blue
4,one,the,#FFE4B5,red
...,...,...,...,...
1391,merry,smaug,#FFE4B5,green
1392,balin,merry,#808000,green
1393,balin,smaug,#808000,green
1394,fili,smaug,#808000,blue


In [221]:
data_df.to_csv('data_df.csv')